# Fitting noisy astronomical data

In the [previous notebook](1. Fitting astronomical images and spectra.ipynb), we had a good time fitting (synthetic) data cubes using a simple model of the continuum + spectral line emission and a simple Gaussian spatial model. We will continue to use the same model, but now we will attempt to fit the data cube in the presence of *heteroscedastic* noise. 

Heteroscedasticity just means that the noise (and therefore uncertainty) is not distributed uniformly and may be correlated. I have generated a synthetic noise profile that varies with channel but is not correlated on spatial scales -- so don't worry about that!

Let's begin by loading the noisy data set and plotting it.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline

# operating system-independent file paths...
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
frequencies_fname = os.path.join(root_dir, 'data', 'frequencies.npy')
noisy_cube_fname = os.path.join(root_dir, 'data', 'noisy_cube.npy')

# loading the data
frequencies = np.load(frequencies_fname)
noisy_cube = np.load(noisy_cube_fname)

# inspect size of data
print(frequencies.shape)
print(noisy_cube.shape)

N_k, N_i, N_j = noisy_cube.shape

In [ ]:
# plot a spectrum through middle pixel
i, j = 16, 16

plt.figure(figsize=(12, 4))
plt.plot(frequencies, noisy_cube[:, i, j])
plt.xlabel('Frequency [GHz]')
plt.ylabel('Spectrum');

Looks reasonable. We can still see a spectral line around 98 GHz, and that there is a positive continuum with amplitude of $\sim 1$.

In [ ]:
# plot a random channel/slice of the cube
k = 130

plt.figure(figsize=(10, 8))
plt.imshow(noisy_cube[k], origin='lower')
plt.colorbar();

## The chi-squared objective function
So we can see right off the bat that fitting these data will be harder. Thankfully, we can leverage a better objective function if we have an estimate of uncertainties, the $\chi^2$ (chi-squared) function:

$$\chi^2 = \sum_{k=1}^{N_k} \frac{\big (y_k - \hat y(x_k; \vec \theta) \big)^2}{\sigma_{y_k}^2}. $$

This objective function is better than using a squared error function because each error term is now scaled by the estimated uncertainty, $\sigma_{y_k}$(or variance $\sigma_{y_k}^2$).
The *reduced* $\chi^2$ is the chi-squared divided by the number of degrees of freedom, $\nu$; in our case, if we have a number of model parameters $N_\theta$, then $\nu = N_k - N_\theta$. Thus, 
$$\chi^2_\nu \equiv \frac{\chi^2}{N_k - N_\theta}.$$  
Conceivably, if we are not overfitting or underfitting our data, and if our **uncertainties are properly estimated**, then $\chi^2_\nu \rightarrow 1$.

This all folds into the $\chi^2$ goodness-of-fit test, which we'll talk about during Session 2.

## Exercise 1

Our first order of business is to make a good estimate of the uncertainties. We want an estimate of the variance, $\sigma_k^2$ at **every channel** $k$. Complete the code below to find these variances, stored in the variable `variances`, which should have length `N_k = 256`. If all goes well, you should see a plot in the cell below like: ![Exercse 1 answer](../../doc/answers/1-2_ex1.png)

In [ ]:
variances = np.zeros((N_k,))

# fill out the code below and find variances
# ------------------------------------------

# hint: You can take the standard deviation along an axis
#   using the `axis` parameter in the `np.nanstd` function.
#   You can simultaneous do this along two dimensions by
#   entering a tuple as the argument, e.g., `axis=(0, 2)`
#   if you want to take it along the zeroth and second
#   dimensions, which will result in an output the same
#   length as the first dimension.


In [ ]:
# plot the spectrum with your newfound uncertainties
uncertainties = np.sqrt(variances)

plt.figure(figsize=(12, 4))
spectrum = noisy_cube[:, 16, 16]
plt.plot(frequencies, spectrum)
plt.fill_between(frequencies, y1=spectrum - uncertainties, y2=spectrum + uncertainties, alpha=0.3)

plt.xlabel('Frequency [GHz]')
plt.ylabel('Spectrum');

## Exercise 2

Now that we have estimated uncertainties, we can compute the $\chi^2$ cost function. Complete the two functions below to evaluate the cost for a spectral model, and we will plot it for you. We will again use the model:

\begin{equation} 
\hat y(\nu; \vec\theta) = F_\nu (\nu; A, \nu_{\rm obs}, \sigma_\nu, K)\\
\ \ \ \ \ \ \ \ \ \ \ = A \Bigg [ \frac{\big (\nu - \nu_{\rm obs} \big)^2}{2 \sigma_\nu^2} \Bigg] + K.
\end{equation}

and the cost function (which is a function of $\nu$, $y$, and $\sigma_{y_k}^2$):
$$\chi^2(\nu, y, \sigma_{y_k}^2) = \sum_{k=1}^{N_k} \frac{\big (y_k - \hat y(x_k; \vec \theta) \big)^2}{\sigma_{y_k}^2}. $$

If you're lazy, you can copy and paste the spectrum model from the previous notebook. Supposing that all goes well, you can expect to see a figure like this:

![Exercise 2 solutions](../../doc/answers/1-2_ex2.png)

In [ ]:
def model_spectrum(x, *parameters):
    """Returns an array of the same length as `x` (given) that 
    produces a model of a spectrum with a flat continuum and
    single spectral emission line.
    """
    
    # unpack the parameters
    line_amplitude, line_frequency, line_sigma, continuum_amplitiude = parameters
    
    model = np.zeros_like(x)
    
    # fill out code below
    # -------------------    
    
    return model

def chi_squared(params, x, y, y_sigma):
    """Given the data and uncertainties and model parameters, 
    calculate the total chi-squared cost (not the reduced
    chi-squared cost).
    """
    
    chi2 = 0
    
    # fill out code below
    # -------------------
    
    
    return chi2

In [ ]:
# we will now compare chi-squared values for a few model examples

model_params_1 = [5, 98, 0.2, 1]
model_params_2 = [0, 98, 1, 1.]
model_params_3 = [4.55, 97.7, 0.1, 1.]

# set up plot
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(12, 12))

# evaluate chi-squared
for i, [params, ax] in enumerate(zip([model_params_1, model_params_2, model_params_3], axes.flat)):
    model = model_spectrum(frequencies, *params)
    chi2 = chi_squared(params, frequencies, spectrum, uncertainties)
    
    # compute reduced chi-squared
    chi2_nu = chi2 / (N_k - len(params))
    
    
    # and plot data
    ax.plot(frequencies, spectrum, label='Data')
    ax.fill_between(frequencies, y1=spectrum - uncertainties, y2=spectrum + uncertainties, alpha=0.3)
    
    # plot model labeled with chi-squared value
    ax.plot(frequencies, model, label=r'Model {:d} ($\chi_\nu^2 = {:.2f}$)'.format(i + 1, chi2_nu))
    
    ax.set_xlabel('Frequency [GHz]')
    ax.set_ylabel('Spectrum')
    ax.legend()

fig.subplots_adjust(hspace=0)
plt.show();

## Exercise 3
### a) Minimizing $\chi^2$

Now that we have intuition about what we're trying to minimize, let's use the `scipy.optimize.minimize` function in order to find the model parameters that minimize $\chi^2$. This is similar to what we've done in the previous notebook. Again, recall that the data and uncertainties should be *arguments* into the minimization routine, but **they are not model parameters which should be varied**.

After you have found the the best-fit model parameters using `scipy.optimize.minimize`, plot a realization of that model against the data. I found the following best-fit parameters:
    
            Line amplitude:      4.192
            Center frequency:   97.696
            Line sigma:          0.126   
            Continuum amplitude: 0.861
Note that if you don't use bounds on $\sigma_\nu$, you may get a negative value. Also, report the **lowest $\chi_\nu^2$** score. I found a minimum of $\chi^2 \approx 237.66$, or $\chi_\nu^2 \approx 0.94$.

Your model should look something like this:
![Exercise 3 solution](../../doc/answers/1-2_ex3a.png)

### b) Residuals
Sometimes it is helpful to visualize your **model residuals**. In the case of fitting a single Gaussian it might be trivial, but often times you will need to compare *multiple models* and select which one is best. The residuals are simply the difference between your data and model at every instance of your independent variable(s) --- in fact, we have used this in the formulation of a cost function.

Use the `matplotlib` tools and other functions to plot the residuals between the *best-fit model* and the spectrum. Compare them with the average uncertainties. Your solution should look (approximately) like this:

![Exercise 3b solution](../../doc/answers/1-2_ex3b.png)

In [ ]:
import scipy.optimize

# part a)
# use scipy.optimize.minimize to find best-fit parameters
# -------------------------------------------------------

# hint: you'll need to supply an initial guess, so feel free
# to use one of the sets of parameters from above


In [ ]:
# part b)
# complete code to find the residuals and plot them
# alongside your estimated uncertainties below
# --------------------------------------------------

# hint: you can use `plt.fill_between` to plot a range
# hint: `plt.axhline` can be used to draw a horizontal line

## Fitting the spatial brightness

At this point, you should know how to fit a one-dimensional model in the presence of noise. Doing the same thing in two dimensions (and even three) is very much analogous to the $1d$ case. We'll begin by fitting the image shown above, channel index 130.

I have provided you with the code for modeling the brightness (which you should have completed in the previous notebook). Please make sure that you understand how the function works before proceeding!

In [ ]:
def model_brightness(grid, *theta):
    """Given a tuple of (ii, jj) in addition to the model parameters
    theta, which consist of the amplitude, center i and j coordinates,
    and sigma, return a circularly symmetric 2-dimensional Gaussian.
    """
    ii, jj = grid
    amp, i0, j0, sigma = theta  
    
    brightness = amp * np.exp(-0.5 * ((ii-i0)**2 + (jj-j0)**2) / sigma**2)
    
    return brightness

In [ ]:
# define a grid over which we want to build our model
ii, jj = np.meshgrid(range(N_i), range(N_j), indexing='ij')

# initialize a model with some parameters and compute residual
k = 130
image = noisy_cube[k]

initial_brightness_params = [5, 15, 17, 3] # don't change!
 
model = model_brightness((ii, jj), *initial_brightness_params)

residuals = image - model

# plot them for comparison
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, [ax1, ax2, ax3] = plt.subplots(1, 3, sharey=True, figsize=(12, 4))

im1 = ax1.imshow(image, origin='lower', cmap='viridis', vmin=-3, vmax=5)
ax1.text(0.05, 0.9, 'Data', size=14, color='black', 
         transform=ax1.transAxes, bbox={'facecolor': 'white'})
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im1, cax=cax1, orientation='vertical')

im2 = ax2.imshow(model, origin='lower', cmap='viridis', vmin=-3, vmax=5)
ax2.text(0.05, 0.9, 'Model', size=14, color='black', 
         transform=ax2.transAxes, bbox={'facecolor': 'white'})
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im2, cax=cax2, orientation='vertical')

im3 = ax3.imshow(residuals, cmap='RdBu_r', vmin=-3, vmax=3, origin='lower')
ax3.text(0.05, 0.9, 'Residuals', size=14, color='black', 
         transform=ax3.transAxes, bbox={'facecolor': 'white'})
divider = make_axes_locatable(ax3)
cax3 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im3, cax=cax3, orientation='vertical');

## Exercise 4
Write a $\chi^2$ cost function for determining the goodness of fit for our spatial brightness model. Remember that when we optimized the spectral model, we also found an estimate of the best-fit peak channel ($\nu_{0} = 97.696$ GHz). Find the channel that this corresponds to, and grab the corresponding image. In my case, the peak channel was number `127`.

(If the channel is a floating-point number, then you'll need to round it to an integer if you want to use it as an index -- you may find integer casting via `int(---)` or `np.rint(---).astype(int)` helpful.)

Use `scipy.minimize.optimize` to find the best-fit parameters for this channel. **You will need to supply an initial guess**; just pick reasonable looking numbers and visualize the data (with a colorbar!) if you need intuition. Plot the data, best-fitting model, and residuals. You should find parameters similar to these:

    Amplitude:     5.51
    i0:           14.52
    j0:           18.36
    Spatial sigma: 4.16
    
    chi-squared: 446.8
    reduced chi2:  1.77

My model comparison looked like this:
![Exercise 4 solution](../../doc/answers/1-2_ex4.png)

In [ ]:
def chi_squared_spatial(theta, ii, jj, image, image_uncertainty):
    """Given parameters and data (i.e., a grid, image (channel), 
    and uncertainty, find the chi-squared value. Note that the 
    uncertainty is a scalar while the image should be a 2d array.
    """
    
    chi2 = 0
    
    # complete the code below
    # -----------------------
    
    return chi2

In [ ]:
frequency_peak = 97.696

k_peak = 0

# fill out code below to find peak channel
# ----------------------------------------



# ----------------------------------------

image = noisy_cube[k_peak]
uncertainty = uncertainties[k_peak]

# minimize chi_squared_spatial and find best-fit parameters
# ---------------------------------------------------------

